## Module 1 Homework: Docker & SQL

### Docker

docker run -it --entrypoint bash python:3.12.8

pip --version

24.3.1

#### Docker networking and docker-compose

services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data

db:5432

#### Trip Segmentation Count

wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

#### Command (for running Postgres and pgAdmin together)

docker-compose up -d

docker-compose down

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"

python ingest_data_trip.py \
  --user=root \
  --password=root \
  --host=localhost \
  --port=5432 \
  --db=ny_taxi \
  --table_name=green_taxi_trips \
  --url=${URL}

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

python ingest_data_zone.py \
  --user=root \
  --password=root \
  --host=localhost \
  --port=5432 \
  --db=ny_taxi \
  --table_name=zones \
  --url=${URL}

#### Longest trip for each day

SELECT
  CASE
    WHEN trip_distance <= 1 THEN 'Up to 1 mile'
    WHEN trip_distance > 1 AND trip_distance <= 3 THEN '1 to 3 miles'
    WHEN trip_distance > 3 AND trip_distance <= 7 THEN '3 to 7 miles'
    WHEN trip_distance > 7 AND trip_distance <= 10 THEN '7 to 10 miles'
    ELSE 'Over 10 miles'
  END AS distance_category,
  COUNT(*) AS trip_count
FROM
  green_taxi_trips
GROUP BY
  distance_category

104,838; 199,013; 109,645; 27,688; 35,202

#### Which was the pick up day with the longest trip distance?

SELECT
  CAST(lpep_pickup_datetime AS DATE),
  trip_distance
FROM
  green_taxi_trips
WHERE
  trip_distance = (SELECT MAX(trip_distance) FROM green_taxi_trips)

2019-10-31

#### Three biggest pickup zones

SELECT
  zpu."Zone",
  SUM(total_amount) AS total_amount_sum
FROM 
  green_taxi_trips t,
  zones zpu
WHERE
  t."PULocationID" = zpu."LocationID"
  AND CAST(lpep_pickup_datetime AS DATE) = '2019-10-18'
GROUP BY
  zpu."Zone"
Having
  SUM(total_amount) > 13000
ORDER BY
  total_amount_sum DESC
LIMIT 3

East Harlem North, East Harlem South, Morningside Heights

#### Largest tip

SELECT
  zdo."Zone",
  MAX(tip_amount) AS tip_amount_max
FROM
  green_taxi_trips t,
  zones zpu,
  zones zdo
WHERE
  t."PULocationID" = zpu."LocationID"
  AND t."DOLocationID" = zdo."LocationID"
  AND zpu."Zone" = 'East Harlem North'
GROUP BY
  zdo."Zone"
ORDER BY
  tip_amount_max DESC
LIMIT 1

### Terraform

terraform init, terraform apply -auto-approve, terraform destroy